In [28]:
import pandas as pd
import numpy as np

# data can be found in DEA

In [29]:
# find the corresponding index which contains str(value) in df
def locate_index(df, value):
    for i in df.index.astype(str):
        if value in i:
            return i

In [30]:
# interpolate cost for a given technology from DEA database sheet
def cal_cost(tech):
    if 'storage' in tech:
        excel = pd.read_excel('inputs/technology_data_catalogue_for_energy_storage.xlsx',\
                              sheet_name=sheet_names[tech],index_col=0,\
                              usecols='B:G',skiprows=[0,1,2])
    elif 'decentral' in tech:
        excel = pd.read_excel('inputs/technologydatafor_heating_installations_marts_2018.xlsx',\
                              sheet_name=sheet_names[tech],index_col=0,\
                              usecols='B:G',skiprows=[0,1])
    else:
        excel = pd.read_excel('inputs/technology_data_for_el_and_dh.xlsx',\
                              sheet_name=sheet_names[tech],index_col=0,\
                              usecols='B:G',skiprows=[0,1,2])
    
    index = locate_index(excel, 'Financial data')
    index_loc_start = excel.index.get_loc(index)
    
    try:        
        index = locate_index(excel, 'Variable O&M')
        index_loc_end = excel.index.get_loc(index)
    
    except KeyError:        
        index = locate_index(excel, 'Fixed O&M')
        index_loc_end = excel.index.get_loc(index)

    df_raw = excel.iloc[index_loc_start+1:index_loc_end+1,:]
    
    df_raw = df_raw.append(excel.loc['Technical lifetime (years)'])
    
    if ('decentral' in tech) & ('storage' not in tech):
        df_raw = df_raw.append(excel.loc['Heat production capacity for one unit (kW)'])
        s = df_raw.loc['Specific investment (1000€/unit)']/df_raw.loc['Heat production capacity for one unit (kW)']
        df_raw.drop(index=df_raw.filter(like='investment',axis=0).index,inplace=True)
        df_raw.loc['investment (M€/MW)'] = s

    if tech in ['OCGT','CCGT','oil','biomass','central resistive heater']:
        index = locate_index(excel, 'efficiency')
        df_raw.loc['efficiency'] = excel.loc[index]/100
    elif 'heat pump' in tech:
        df_raw.loc['efficiency'] = 4
    else:
        df_raw.loc['efficiency'] = 1
    
    df_raw = df_raw.loc[:,excel.columns.isin(years)]
    
    df_raw = df_raw.groupby(df_raw.index).sum(axis=0)
    
    df_raw.replace(to_replace='-',value=np.nan,inplace=True)

    df = pd.DataFrame(index=df_raw.index,columns=years)

    for index in df.index:
        
        values = np.interp(x=years,xp=df_raw.columns.values,fp=df_raw.loc[index,:].values.astype(float))
        
        df.loc[index,:] = values
        
    return df

In [31]:
sheet_names = {'onwind': '20 Onshore turbines',
               'offwind': '21 Offshore turbines',
               'OCGT': '52 OCGT - Natural gas',
               'CCGT': '05 Gas turb. CC, Back-pressure',
               'oil': '50 Diesel engine farm',
               'biomass CHP': '09 Straw, Large',
               'biomass HOP': '09 Straw HOP',
               'central coal CHP': '01 Coal CHP',
               'central heat pump': '40 Comp. heat pump, DH',
               'central resistive heater': '41 Electric Boilers',
               'central gas boiler': '44 Natural Gas DH Only',
               'decentral gas boiler': '202 Natural gas boiler',
               'decentral heat pump': '207.7 Ground source existing',
               'decentral resistive heater': '216 Electric heating',
               'central water tank storage': '140 PTES seasonal',
#                'decentral water tank storage': '142 Small scale hot water tank',
               'hydrogen storage underground': '151c Hydrogen Storage - Caverns',
               'hydrogen storage tank': '151a Hydrogen Storage - Tanks',
#                'battery storage': '180 Lithium Ion Battery',
              }

usecols = {}

years = np.arange(2020,2055,5)

In [32]:
d_by_tech = {}

# calculate costs and save results as a dict of df, indexed by cost, columned by year
for tech in sheet_names.keys():
    
    print(tech+' in PyPSA corresponds to '+sheet_names[tech]+' in DEA database.')
    
    df = cal_cost(tech=tech)
    
    df.fillna(value=0,inplace=True)
        
    d_by_tech[tech] = df

onwind in PyPSA corresponds to 20 Onshore turbines in DEA database.
offwind in PyPSA corresponds to 21 Offshore turbines in DEA database.
OCGT in PyPSA corresponds to 52 OCGT - Natural gas in DEA database.
CCGT in PyPSA corresponds to 05 Gas turb. CC, Back-pressure in DEA database.
oil in PyPSA corresponds to 50 Diesel engine farm in DEA database.
biomass CHP in PyPSA corresponds to 09 Straw, Large in DEA database.
biomass HOP in PyPSA corresponds to 09 Straw HOP in DEA database.
central coal CHP in PyPSA corresponds to 01 Coal CHP in DEA database.
central heat pump in PyPSA corresponds to 40 Comp. heat pump, DH in DEA database.
central resistive heater in PyPSA corresponds to 41 Electric Boilers in DEA database.
central gas boiler in PyPSA corresponds to 44 Natural Gas DH Only in DEA database.
decentral gas boiler in PyPSA corresponds to 202 Natural gas boiler in DEA database.
decentral heat pump in PyPSA corresponds to 207.7 Ground source existing in DEA database.
decentral resistive

In [33]:
# aggregate technologies into a dict, whose keys are years
d_by_year = {}

for year in years:
    
    index = ['investment','FOM','VOM','lifetime','efficiency']
    
    df = pd.DataFrame(index=index,columns=sheet_names.keys(),data=0,dtype=float)

    for tech in sheet_names.keys():

        index = locate_index(d_by_tech[tech], 'investment')
        
        if ('M€' in index) & ('MW' in index):
            CC = d_by_tech[tech].at[index,year]*1e6 # convert from MEUR/MW(h) to EUR/MW(h)
        elif ('M€' in index) & ('GW' in index):
            CC = d_by_tech[tech].at[index,year]*1e3 # convert from MEUR/GW(h) to EUR/MW(h)
        elif ('€' in index) & ('kW' in index):
            CC = d_by_tech[tech].at[index,year]*1e3 # convert from EUR/KW(h) to EUR/MW(h)

        df.at['investment',tech] = CC/1e3 # in EUR/KW

        index = locate_index(d_by_tech[tech], 'Fixed O&M')

        FOM = d_by_tech[tech].at[index,year] # in EUR/MW/year
        if tech == 'decentral water tank storage':
            FOM = d_by_tech[tech].at[index,year]/3 # from EUR/tank/year to EUR/MW/year
        elif tech == 'hydrogen storage underground': # I believe there is an error in DEA database
            FOM = CC*0.02
            
        df.at['FOM',tech] = np.round(FOM/CC*100,3) # in %/year

        index = locate_index(d_by_tech[tech], 'Variable O&M')

        try:
            VOM = d_by_tech[tech].at[index,year] # in EUR/MWh
        except KeyError:
            VOM = 0

        df.at['VOM',tech] = VOM
        
        df.at['lifetime',tech] = d_by_tech[tech].at['Technical lifetime (years)',year]
        
        df.at['efficiency',tech] = d_by_tech[tech].at['efficiency',year]
        
    d_by_year[year] = df

# Other sources

In [34]:
# onwind, offwind, OCGT, CCGT, biomass electricity, central heat pump are from DEA
source_DEA = 'Technology Data for Energy Plants for Electricity and District heating generation'

# solar utility from Vartiaian 2019
data = np.interp(x=years,xp=[2020, 2030, 2040, 2050],fp=[431, 275, 204, 164])

solar_uti = pd.Series(data=data,index=years)

source_Vartiainen = 'Impact of weighted average cost of capital, capital expenditure, and other parameters on future utility‐scale PV levelised cost of electricity'

# solar rooftop from ETIP 2019
data = np.interp(x=years,xp=[2020, 2030, 2050],fp=[1150, 800, 550])

solar_roof = pd.Series(data=data,index=years)

source_ETIP = 'European PV Technology and Innovation Platform'

# nuclear, coal, lignite from Lazards
source_Lazards = 'Lazard’s Levelized Cost of Energy Analysis - Version 13.0'

# Overwrite the cost assumption of 2030 for other years

In [35]:
unit_to_replace = {'investment':'EUR/kW(h)',
                   'lifetime':'years',
                   'FOM':'%/year',
                   'efficiency':'per unit',
                   'VOM':'EUR/MWh',
                  }

In [36]:
for year in years:
    df = d_by_year[year]
    costs = pd.read_csv('inputs/costs_PyPSA.csv', index_col=list(range(2))).sort_index()
    
    # biomass is differentiated by biomass CHP and HOP
    costs.drop(index='biomass',inplace=True)
    
    # central heat pump for district heating from DEA
    costs.rename({'central air-sourced heat pump':'central heat pump'},inplace=True)
    costs.rename({'decentral air-sourced heat pump':'decentral heat pump'},inplace=True)
    costs.drop(index='decentral ground-sourced heat pump',inplace=True)
    
    # central CHP is gas-fired 
    costs.rename({'central CHP':'central gas CHP'},inplace=True)
    
    # hydrogen storage now have two forms
    costs.drop(index='hydrogen storage',inplace=True)
    
    for tech in df.columns:
        for para in df.index:
            costs.at[(tech, para), 'value'] = df.at[para, tech]
            costs.at[(tech, para), 'source'] = source_DEA
            
    # solar utility from Vartiaian 2019
    costs.loc[('solar-utility','investment'),'value'] = solar_uti[year]
    costs.loc[('solar-utility','investment'),'source'] = source_Vartiainen
    
    costs.loc[('solar-utility','lifetime'),'value'] = 30
    costs.loc[('solar-utility','lifetime'),'source'] = source_Vartiainen
    
    # solar rooftop from ETIP 2019
    costs.loc[('solar-rooftop','investment'),'value'] = solar_roof[year]
    costs.loc[('solar-rooftop','investment'),'source'] = source_ETIP
    
    costs.loc[('solar-rooftop','lifetime'),'value'] = 30
    costs.loc[('solar-rooftop','lifetime'),'source'] = source_ETIP
    
    # nuclear from Lazards
    costs.loc[('nuclear','investment'),'value'] = 8595
    costs.loc[('nuclear','investment'),'source'] = source_Lazards
    
    costs.loc[('nuclear','FOM'),'value'] = 1.4
    costs.loc[('nuclear','FOM'),'source'] = source_Lazards
    
    costs.loc[('nuclear','VOM'),'value'] = 3.5
    costs.loc[('nuclear','VOM'),'source'] = source_Lazards
    
    costs.loc[('nuclear','efficiency'),'value'] = 0.33
    costs.loc[('nuclear','efficiency'),'source'] = source_Lazards
    
    costs.loc[('nuclear','fuel'),'value'] = 2.6
    costs.loc[('nuclear','fuel'),'source'] = source_Lazards
    costs.loc[('uranium','fuel'),'value'] = 2.6
    costs.loc[('uranium','fuel'),'source'] = source_Lazards
    
    costs.loc[('nuclear','lifetime'),'value'] = 40
    costs.loc[('nuclear','lifetime'),'source'] = source_Lazards
    
    # coal from Lazards and BP 2019
    costs.loc[('coal','investment'),'value'] = 4162.5
    costs.loc[('coal','investment'),'source'] = source_Lazards
    
    costs.loc[('coal','FOM'),'value'] = 1.6
    costs.loc[('coal','FOM'),'source'] = source_Lazards
    
    costs.loc[('coal','VOM'),'value'] = 3.5
    costs.loc[('coal','VOM'),'source'] = source_Lazards
    
    costs.loc[('coal','efficiency'),'value'] = 0.33
    costs.loc[('coal','efficiency'),'source'] = source_Lazards
    
    costs.loc[('coal','fuel'),'value'] = 8.15
    costs.loc[('coal','fuel'),'source'] = 'BP 2019'
    
    costs.loc[('coal','lifetime'),'value'] = 40
    costs.loc[('coal','lifetime'),'source'] = source_Lazards
    
    # lignite from Lazards and DIW
    costs.loc[('lignite','investment'),'value'] = 4162.5
    costs.loc[('lignite','investment'),'source'] = source_Lazards
    
    costs.loc[('lignite','FOM'),'value'] = 1.6
    costs.loc[('lignite','FOM'),'source'] = source_Lazards
    
    costs.loc[('lignite','VOM'),'value'] = 3.5
    costs.loc[('lignite','VOM'),'source'] = source_Lazards
    
    costs.loc[('lignite','efficiency'),'value'] = 0.33
    costs.loc[('lignite','efficiency'),'source'] = source_Lazards
    
    costs.loc[('lignite','fuel'),'value'] = 2.9
    costs.loc[('lignite','fuel'),'source'] = 'DIW'

    costs.loc[('lignite','lifetime'),'value'] = 40
    costs.loc[('lignite','lifetime'),'source'] = source_Lazards
      
    # replace unit with value of 0 to a proper unit
    costs.unit.values[costs.unit.isna()] = (costs.index.get_level_values(1)[costs.unit.isna()]).map(unit_to_replace)
    
    # save the cost assumption per year
    costs.to_csv('outputs/costs_{}.csv'.format(year))

/home/kun/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [37]:
costs.filter(like='investment',axis=0)

,,value,unit,source
technology,parameter,,,
CCGT,investment,1100.000000,EUR/kWel,Technology Data for Energy Plants for Electric...
DAC,investment,250.000000,EUR/(tCO2/a),Fasihi/Climeworks
HVAC overhead,investment,400.000000,EUR/MW/km,Hagspiel
HVDC inverter pair,investment,150000.000000,EUR/MW,Hagspiel
HVDC overhead,investment,400.000000,EUR/MW/km,Hagspiel
HVDC submarine,investment,2000.000000,EUR/MW/km,DTU report based on Fig 34 of https://ec.europ...
OCGT,investment,411.840000,EUR/kWel,Technology Data for Energy Plants for Electric...
PHS,investment,2000.000000,EUR/kWel,DIW DataDoc http://hdl.handle.net/10419/80348
battery inverter,investment,411.000000,USD/kWel,budischak2013
